In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import torch
import torch.nn as nn
import numpy as np
from torchvision import models, transforms
import pretrainedmodels
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from data import data_reader

Tensor = torch.Tensor
rand_m = np.random.random

In [2]:
MODEL_NAME = 'xception_gcn'
if not os.path.exists('weights/'):
    !mkdir weights
if not os.path.exists('weights/{}'.format(MODEL_NAME)):
    !mkdir weights/{MODEL_NAME}

In [3]:
from models import model_xception
model = model_xception(6, use_gcn=True).to('cuda')
optim = torch.optim.Adam(model.parameters())
loss_func = torch.nn.CrossEntropyLoss()

In [4]:
from data import data_reader
class args:
    bs = 16
    path = 'train2000/'

x_norm = transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                  std=[0.5, 0.5, 0.5])
trm = transforms.Compose([
    transforms.Resize(299),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    x_norm
])

val_trm = transforms.Compose([
    transforms.Resize(333),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    x_norm
])

dr = data_reader(args, 'train2000/train2000.csv', trm=trm, val_trm=val_trm)

train_loader, test_loader = dr.get_train_loader()

In [5]:
from utils.train_model import trainer
model_trainer = trainer(model, epoch=100, train_loader=train_loader, test_loader=test_loader, 
        optim=optim, loss_func=loss_func, model_name='xception_gcn')

In [ ]:
model_trainer.train()

Epoch 0 | Batch # 0 Train Loss 1.79491 
Epoch 0 | Batch # 1 Train Loss 38.97139 
Epoch 0 | Batch # 2 Train Loss 1047.56116 
Epoch 0 | Batch # 3 Train Loss 756.50763 
Epoch 0 | Batch # 4 Train Loss 491.84625 
Epoch 0 | Batch # 5 Train Loss 926.57745 
Epoch 0 | Batch # 6 Train Loss 717.58691 
Epoch 0 | Batch # 7 Train Loss 328.11438 
Epoch 0 | Batch # 8 Train Loss 221.27597 
Epoch 0 | Batch # 9 Train Loss 618.04797 
Epoch 0 | Batch # 10 Train Loss 1269.71045 
Epoch 0 | Batch # 11 Train Loss 544.12842 
Epoch 0 | Batch # 12 Train Loss 370.44424 
Epoch 0 | Batch # 13 Train Loss 485.84854 
Epoch 0 | Batch # 14 Train Loss 168.67628 
Epoch 0 | Batch # 15 Train Loss 137.81131 
Epoch 0 | Batch # 16 Train Loss 98.04649 
Epoch 0 | Batch # 17 Train Loss 146.13324 
Epoch 0 | Batch # 18 Train Loss 66.00172 
Epoch 0 | Batch # 19 Train Loss 97.76830 
Epoch 0 | Batch # 20 Train Loss 113.80906 
Epoch 0 | Batch # 21 Train Loss 37.65506 
Epoch 0 | Batch # 22 Train Loss 127.82623 
Epoch 0 | Batch # 23 Train